In [ ]:
import pandas as pd
import os 
import numpy as np
import re
import random
import tomotopy as tp
import sys
import pickle
import time

In [ ]:
input_path = "abstracts_processed.csv"
with open(input_path, "rb") as fp:   
    # Unpickling
    documents = pickle.load(fp)

# Experiment 1 - HDP

In [ ]:
## Setting the hyperparameters
tw = tp.TermWeight.ONE # term weighting scheme in TermWeight. The default value is TermWeight.ONE
initial_k = 2
min_cf=3 # minimum collection frequency of words. Words with a smaller collection frequency than min_cf are excluded from the model. The default value is 0, which means no words are excluded.
min_df=0 # minimum document frequency of words. Words with a smaller document frequency than min_df are excluded from the model. The default value is 0, which means no words are excluded
rm_top=5 # the number of top words to be removed. If you want to remove too common words from model, you can set this value to 1 or more. The default value is 0, which means no top words are removed.
alpha = 0.1 # hyperparameter of Dirichlet distribution for document-topic
eta = 0.01 # hyperparameter of Dirichlet distribution for topic-word
gamma = 0.1 # concentration coeficient of Dirichlet Process for table-topic
transform = None # a callable object to manipulate arbitrary keyword arguments for a specific topic model
seed = 41 # random seed
model_burn_in = 500 
train_updates = 10000
train_iter = 10
save_path = "hdp_model.bin" #.bin format

In [ ]:
model = tp.HDPModel(tw=tw, min_cf=min_cf, min_df=min_df, rm_top=rm_top, initial_k=initial_k, alpha=alpha, 
                    eta=eta, gamma=gamma, transform=transform)

In [ ]:
# adding documents to the model 
for doc in documents: model.add_doc(doc)

In [ ]:

start = time.time()
# training**
model.burn_in = model_burn_in
# initialising 
model.train(iter=0)
print('Num docs:', len(model.docs), ', Vocab size:', len(model.used_vocabs), ', Num words:', model.num_words)
print('Removed top words:', model.removed_top_words)
print('Training...', file=sys.stderr, flush=True)
# actual training 
t = []
LLs = []
for i in range(0, train_updates, train_iter):
    model.train(train_iter)
    if i%1000==0:print('Iteration: {}'.format(i))
    t.append(i)
    LLs.append(model.ll_per_word)

end = time.time()
print("Time elapsed: "+ str(round(end - start,1))+" s")

In [ ]:
model.summary()

In [ ]:
def train_HDP()

# Experiment 2 - Online HDP

In [ ]:
from gensim.models import HdpModel
from gensim.corpora import Dictionary
random.seed = 11 #set the seed right away

In [ ]:
# Train and test (80-20)
print(len(documents))
set1_size = int(0.3*len(documents)) 
set2_size = int(0.7*len(documents)) 

random.shuffle(documents)

set1_docs = documents[0:set1_size]
set2_docs = documents[set1_size:]

len(set1_docs) + len(set2_docs) == len(documents)

In [ ]:
# Preparing the input in gensim format 
from copy import deepcopy

def prepare_gensim_input(docs, old_dict=None):
    # TODO: add possibility to remove extremes (too frequent/rare words)
    dictionary = Dictionary(docs)
    if old_dict is not None: 
        old_dict_copy = deepcopy(old_dict)
        old_dict_copy.merge_with(dictionary)
        dictionary = old_dict_copy
    corpus = [dictionary.doc2bow(doc, allow_update=True) for doc in docs] # bag of words corpus 
    return dictionary, corpus

In [ ]:
dictionary1, corpus1 = prepare_gensim_input(set1_docs)

### OHDP model

Online HDP provides <u>the speed of online variational Bayes with the modeling flexibility of the HDP</u>.<br>
The idea behind Online variational Bayes in general is to optimize the variational objective function with stochastic optimization.<br>
The challenge we face is that the existing coordinate ascent variational Bayes algorithms for the HDP require complicated approximation methods or numerical optimization. 


Look [here](https://radimrehurek.com/gensim/models/hdpmodel.html) for more.

In [ ]:
# HYERARAMETERS 
alpha = 1 #(int, optional) – Second level concentration - below one leads to  more sparse solutions
gamma = 1 #(int, optional) – First level concentration
eta = 0.01 #(float, optional) – The topic Dirichlet

In [ ]:
start = time.time()
hdp = HdpModel(corpus1, dictionary1, max_chunks=None, 
               max_time=None, chunksize=256, kappa=1.0, tau=64.0, 
               K=15, T=150, alpha=alpha, gamma=gamma, eta=eta, scale=1.0, 
               var_converge=0.0001, outputdir=None, random_state=None)
end = time.time()

print("Time elapsed: "+ str(round(end - start,1))+" s")

In [ ]:
len(hdp.get_topics())

Print 20 topics with top 10 most probable words

In [ ]:
from gensim.parsing.preprocessing import preprocess_string, strip_punctuation, strip_numeric

def print_hdp_topics(model, num_words):
    hdp_topics = model.show_topics(num_topics=200, num_words=num_words)

    topics = []
    filters = [lambda x: x.lower(), strip_punctuation, strip_numeric]
    
    print("Number of topics found: "+ str(len(hdp_topics)))
    print("")
    
    for idx, topic in enumerate(hdp_topics):
        print("Topic "+str(idx)+" -------")
        t = preprocess_string(topic[1], filters)
        print(" - ".join(t))
        print("")
        topics.append(t)

In [ ]:
print_hdp_topics(hdp, num_words=20)

### In depth exploration of results 

In [ ]:
def doc2topics(topics_document, num_topics):
    """ 
        topics_document : list of tuples (topic_id,weight)
        
    This function creates one row of the documents2topics matrix by 
    extracting all the topics relative to the input document. """
    
    res = pd.DataFrame(columns=range(num_topics))
    for topic_weight in topics_document[1:]:
        res.loc[0, topic_weight[0]] = topic_weight[1]
    return res

def docs2topics(model, corpus):
    """ Builds the docs2topics matrix. """
    K = len(hdp.get_topics()) # get number of topics
    topicsfordocs = [model[doc] for doc in corpus]
    matrix = pd.concat([doc2topics(doc, K) for doc in topicsfordocs]).reset_index(drop=True).fillna(0)
    return matrix

In [ ]:
documents2topics = docs2topics(hdp, corpus1)

In [ ]:
documents2topics.head(3)

In [ ]:
pd.DataFrame.sum(documents2topics, axis=1)

In [ ]:
# Which document are about topic 15
documents2topics.sort_values(15, ascending=False)[15].head(10)

In [ ]:
documents2topics.loc[documents2topics.idxmax(axis=1).sort_values().index]

In [ ]:
# number of nonzeros in the rows 
np.mean(documents2topics.astype(bool).sum(axis=1))

In [ ]:
# number of nonzeros in the columns 
print(np.mean(documents2topics.astype(bool).sum(axis=0)))
print(np.min(documents2topics.astype(bool).sum(axis=0)))
print(np.max(documents2topics.astype(bool).sum(axis=0)))

In [ ]:
%matplotlib inline
import seaborn as sns; 
sns.set(rc={'figure.figsize':(5,10)})
sns.heatmap(documents2topics.loc[documents2topics.idxmax(axis=1).sort_values().index])

### Filtering the topics 

**Idea**: filter out "bad" topics, so as to retain only the topics pertinent to the collection. <br>
Definition of bad: topics whose max weight across all the documents is below a tbd threshold.

**Also**: we could try filter out the topics that are "too general", where the measure of generality is a function of the number of documents they appear and their respective weights there. 

**Note**: we probably need to re-adjust mixing weights after the filtering.

In [ ]:
sum(documents2topics.max(axis=0).sort_values() == 0) # number of topics not appearing in ANY document 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(5,5))
h= plt.hist(documents2topics.max(axis=0).sort_values(), bins=20, density=True, cumulative=True) 
plt.show(h)

In [ ]:
plt.figure(figsize=(5,5))
h= plt.hist(documents2topics.astype(bool).sum(axis=0)) 
plt.show(h)

In [ ]:
documents2topics.loc[:,(documents2topics.astype(bool).sum(axis=0) > 400)]

### Updating the model - online step

In [ ]:
dictionary2, corpus2 = prepare_gensim_input(set2_docs, old_dict=dictionary1)
# hdp.evaluate_test_corpus(test_corpus) - Returns the value of total likelihood obtained by evaluating the model for all documents in the test corpus.

In [ ]:
hdp_new = deepcopy(hdp)

# update the dictionary
hdp_new.id2word = dictionary2

# the m_lambda matrix is a (topic x word) matrix 
# we need to add a new column for each new word before updating the model 
new_words = len(dictionary2)-len(dictionary1) #the new words
# we initialise the new words with weight 0 in the model
m_lambda_new = np.hstack([hdp.m_lambda,np.zeros((150, new_words))])
# TODO: check this with the gensim guys 
hdp_new.m_lambda = m_lambda_new

# the m_Elogbeta matrix is a (topic x word) matrix 
m_Elogbeta_new = np.hstack([hdp.m_Elogbeta,np.zeros((150, new_words))])
hdp_new.m_Elogbeta = m_Elogbeta_new

# the m_timestamp vector is a (wordx1) matrix that 'Helps to keep track and perform lazy updates on lambda.'
m_timestamp_new = np.hstack([hdp.m_timestamp,np.zeros((new_words))]).astype(int)
hdp_new.m_timestamp = m_timestamp_new

In [ ]:
hdp_new.update(corpus2)

In [ ]:
print_hdp_topics(hdp_new, num_words=10)

In [ ]:
# compute the new topics distribution for the whole collection 
_,corpus_full = prepare_gensim_input(documents, old_dict=dictionary2)
documents2topics_new = docs2topics(hdp_new, corpus_full)

In [ ]:
sum(documents2topics_new.max(axis=0).sort_values() == 0) # number of topics not appearing in ANY document (second corpus)

In [ ]:
plt.figure(figsize=(5,5))
h= plt.hist(documents2topics_new.max(axis=0).sort_values(), bins=20, density=True, cumulative=True) 
plt.show(h)

In [ ]:
# let's have a closer look at the differences btw the old and the new topics distributions 
documents2topics_new.loc[1,:].sort_values(ascending=False)

In [ ]:
hdp_new.print_topic(3)

In [ ]:
documents2topics.loc[0,:].sort_values(ascending=False)

In [ ]:
hdp.print_topic(112)